In [2]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d pdavpoojan/the-rvlcdip-dataset-test

Dataset URL: https://www.kaggle.com/datasets/pdavpoojan/the-rvlcdip-dataset-test
License(s): unknown
100% 3.60G/3.62G [00:53<00:00, 106MB/s]
100% 3.62G/3.62G [00:53<00:00, 72.4MB/s]


In [7]:
!unzip '/content/the-rvlcdip-dataset-test.zip'

Streaming output truncated to the last 5000 lines.
  inflating: test/scientific_publication/PUBLICATIONS040137-0.tif  
  inflating: test/scientific_publication/PUBLICATIONS041468-1.tif  
  inflating: test/scientific_publication/PUBLICATIONS041570-1.tif  
  inflating: test/scientific_publication/PUBLICATIONS041737-1.tif  
  inflating: test/scientific_publication/PUBLICATIONS042046-2.tif  
  inflating: test/scientific_publication/PUBLICATIONS042251A-.tif  
  inflating: test/scientific_publication/PUBLICATIONS042455-2.tif  
  inflating: test/scientific_publication/PUBLICATIONS044016-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044594-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044641-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044730-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044895-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS045235-5.tif  
  inflating: test/scientific_publication/PUBLICATIONS045310-5.tif

In [20]:
!pip install torch
!pip install torchvision
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pillow
!pip install opencv-python
!pip install pandas
!pip install numpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,757 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [18]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import pytesseract
import cv2
import numpy as np
import pandas as pd
import os
import glob

In [21]:
import pytesseract
print(pytesseract.image_to_string('/content/test/invoice/0000037007.tif'))  # Replace with your image path

SPRTION ORDER C C

(ULICH-KOENIG ADVERTISING INC. 1451 SO. MIAMI AVENUE, MIAMI, FLORIDA 33131
5-971-0451 905-371-9801 .

3/29/78

 
 

Mr. DeWitt Ingraham
WEDR-Radio
1437 NW 35th Streot

+ Miami, Plorida :

$18 YOUR AUTHORITY To INSERT THE ADVERTISING 01

 

 

 

 

  

 

a Radio Commercials Gross Cost
330
we Poo} gm an

 

$275.28

0994.50089

 

Bill Murphy

ren phy

 

 

 



In [10]:
# Define a simple ResNet-based image enhancement model
class ImageEnhancementModel(nn.Module):
    def __init__(self):
        super(ImageEnhancementModel, self).__init__()
        # Using a simplified ResNet architecture for image enhancement
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 1, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out + identity  # Skip connection

In [11]:
def enhance_image_quality(image):
    """
    Enhance image quality using various CV techniques
    """
    # Convert to numpy array if needed
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(image.astype(np.uint8))

    # Apply bilateral filter to reduce noise while preserving edges
    denoised = cv2.bilateralFilter(enhanced, 9, 75, 75)

    # Sharpen the image
    kernel = np.array([[-1,-1,-1],
                      [-1, 9,-1],
                      [-1,-1,-1]])
    sharpened = cv2.filter2D(denoised, -1, kernel)

    return sharpened

In [12]:
def preprocess_image(image_path, output_path=None):
    """
    Enhanced preprocessing pipeline
    """
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Initial CV2 preprocessing
    # Apply Otsu's thresholding
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Remove noise using morphological operations
    kernel = np.ones((2,2), np.uint8)
    denoised = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Enhance image quality
    enhanced = enhance_image_quality(denoised)

    # Deskew the image if needed
    coords = np.column_stack(np.where(enhanced > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = 90 + angle
    (h, w) = enhanced.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(enhanced, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # Save if output path provided
    if output_path:
        cv2.imwrite(output_path, rotated)

    return rotated

In [13]:
def configure_tesseract():
    """
    Configure Tesseract with optimal parameters for document processing
    """
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,()-:$/\% '
    return custom_config

In [14]:
def extract_text_from_image(image_path):
    """
    Enhanced text extraction with configured Tesseract
    """
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Convert to PIL Image
    pil_image = Image.fromarray(processed_image)

    # Get Tesseract configuration
    custom_config = configure_tesseract()

    # Extract text with confidence scores
    try:
        # Get detailed OCR data
        ocr_data = pytesseract.image_to_data(pil_image, config=custom_config, output_type=pytesseract.Output.DICT)

        # Filter out low confidence text
        filtered_text = []
        for i, conf in enumerate(ocr_data['conf']):
            if float(conf) > 60:  # Confidence threshold
                text = ocr_data['text'][i]
                if text.strip():  # Only add non-empty text
                    filtered_text.append(text)

        extracted_text = ' '.join(filtered_text)
        return extracted_text
    except Exception as e:
        print(f"Error in OCR process: {e}")
        return ""

In [15]:
def process_directory(input_dir, output_dir):
    """
    Process all images in a directory
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Process all images
    results = []
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"processed_{filename}")

            try:
                # Extract text
                extracted_text = extract_text_from_image(input_path)

                # Store results
                results.append({
                    'filename': filename,
                    'extracted_text': extracted_text,
                    'processed_image_path': output_path
                })

                print(f"Successfully processed {filename}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Create DataFrame and save results
    df = pd.DataFrame(results)
    df.to_csv(os.path.join(output_dir, "extraction_results.csv"), index=False)

    return df

In [22]:
# Main execution
if __name__ == "__main__":
    input_directory = "/content/test/invoice"  # Update with your input directory
    output_directory = "/content/processed_invoices"  # Update with your output directory

    # Process all images
    results_df = process_directory(input_directory, output_directory)
    print("Processing complete. Results saved to extraction_results.csv")

Successfully processed 2028702721.tif
Successfully processed CTRCONTRACTS011579-1.tif
Successfully processed 2028718490.tif
Successfully processed 2072484360.tif
Successfully processed 0001233882.tif
Successfully processed 2048406835.tif
Successfully processed 91514715.tif
Successfully processed 2028718578.tif
Successfully processed 2049433283.tif
Successfully processed 2070910600.tif
Successfully processed 507484723.tif
Successfully processed 00922266_00922271.tif
Successfully processed 0000218219.tif
Successfully processed 0060067614.tif
Successfully processed 2030402746.tif
Successfully processed 2029377237.tif
Successfully processed 0012192413.tif
Successfully processed 2072654498.tif
Successfully processed 2074104174.tif
Successfully processed 0000173220.tif
Successfully processed 0060046962.tif
Successfully processed 87147931_87147932.tif
Successfully processed 502197662.tif
Successfully processed 2063576176.tif
Successfully processed 512982282_512982287.tif
Successfully processe